# Instruction

**Run each cell one by one**

# Start

1. Load the dataset from the google drive

In [1]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pprint, pickle
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id_1_train="13N5Il4V77Od7rK5ywff7YBWDD9PGJBdT"
id_1_evaluation="1RFAezNqq00h8lYs1f_kywJ0lDLhx-sKG"

id_2_train="17YZy5p42Bprj2Bo19lpH04iIBgf8J42l"
id_2_evaluation="13ZuGeuCSopUsPDMUz5TRZfaSuGnA_kZ7"

id_3_train="1mRJkhas7St256PhJPUI4TDQN2ENA2RnO"
id_3_evaluation="1fyk1WmHIk3V_XCDUch28DwWhHXFitogM"

id_4_train="1uEn_rGZMGLEcYuIfrXBcYroAOKTvkVRw"
id_4_evaluation="1jnp1LOgUcZhzcB7weaEQ3FTGk2gNGHcc"


id_5_train="1AGmGhS6pH5ED5v1Qc8RyJbHdybKG4G96"
id_5_evaluation="1IC74vwV5_IyBvBFfSpqIACzgYOI-2i4u"

id_6_train="1Td_zmiqkpKNQBwBje7pyJoRde9wUjABh"
id_6_evaluation="1XiJcjN1-Pe7-R89PugtFspWilF3QOlqi"

id_7_train="1dujTQPIGDBSozlbXt_hJj_qKIftxWVka"
id_7_evaluation="1LniXihHiqx1PSlvmJXu20LaOeXCjBmeQ"


id_8_train="1T66t4HpF1XTB6QNL9ZdwILWGU5sEmf09"
id_8_evaluation="1lxZVBtjtdzF7aZPx4oh2265m-049XgnO"

id_9_train="1zGIpgC3RNRb1qF8t-id3IGtvd92C7IQV"
id_9_evaluation="1guGz6wBBwzTrRNQGCvP-ar2GuI8ZQWe7"

def load(id_train,id_evaluation,subject):
  downloaded = drive.CreateFile({'id':id_train}) 
  downloaded.GetContentFile('B0'+str(subject)+'T.mat')
  downloaded = drive.CreateFile({'id':id_evaluation}) 
  downloaded.GetContentFile('B0'+str(subject)+'E.mat')

load(id_1_train,id_1_evaluation,1)
load(id_2_train,id_2_evaluation,2)
load(id_3_train,id_3_evaluation,3)

load(id_4_train,id_4_evaluation,4)
load(id_5_train,id_5_evaluation,5)
load(id_6_train,id_6_evaluation,6)

load(id_7_train,id_7_evaluation,7)
load(id_8_train,id_8_evaluation,8)
load(id_9_train,id_9_evaluation,9)

2. Load package

In [2]:
import numpy as np
import scipy.io as sio
from tensorflow.keras.utils import to_categorical
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd.function import Function
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader,TensorDataset
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plot
from sklearn.decomposition import PCA
import torch.utils.data as Data
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from torchvision import models
from torchsummary import summary
import torch.optim.lr_scheduler as lr_scheduler


3. Convert the data in mat format into Numpy array


In [3]:
def load_all_data (crossValidation, data_path): 

    big_X_train, big_y_train, big_X_test, big_y_test = [None]*9, [None]*9, [None]*9, [None]*9
    for subject in range (0,9):
        path = data_path+'s' + str(subject+1) + '/'
        big_X_train[subject], big_y_train[subject] = get_data(subject+1, True ,path)
        big_X_test[subject], big_y_test[subject] = get_data(subject+1, False ,path)
    
    return big_X_train, big_y_train, big_X_test, big_y_test

def get_data(subject,training,path, highpass = False):
	'''	Loads the dataset 2a of the BCI Competition IV
	available on http://bnci-horizon-2020.eu/database/data-sets

	Keyword arguments:
	subject -- number of subject in [1, .. ,9]
	training -- if True, load training data
				if False, load testing data
	
	Return:	data_return 	numpy matrix 	size = NO_valid_trial x 22 x 1750
			class_return 	numpy matrix 	size = NO_valid_trial
	'''
	NO_channels = 3
	NO_tests = 120+120+160	
	Window_Length = 7*250 

	class_return = np.zeros(NO_tests)
	data_return = np.zeros((NO_tests,NO_channels,Window_Length))

	NO_valid_trial = 0
	if training:
		a = sio.loadmat(path+'B0'+str(subject)+'T.mat')
	else:
		a = sio.loadmat(path+'B0'+str(subject)+'E.mat')
	a_data = a['data']
	for ii in range(0,a_data.size):
		a_data1 = a_data[0,ii]
		a_data2= [a_data1[0,0]]
		a_data3= a_data2[0]
		a_X 		= a_data3[0]
		a_trial 	= a_data3[1]
		a_y 		= a_data3[2]
		a_fs 		= a_data3[3]
		a_classes 	= a_data3[4]
		a_artifacts = a_data3[5]
		a_gender 	= a_data3[6]
		a_age 		= a_data3[7]

		for trial in range(0,a_trial.size):
			if(a_artifacts[trial]==0):
				data_return[NO_valid_trial,:,:] = np.transpose(a_X[int(a_trial[trial]):(int(a_trial[trial])+Window_Length),:3])
				class_return[NO_valid_trial] = int(a_y[trial])
				NO_valid_trial +=1


	return data_return[0:NO_valid_trial,:,:], class_return[0:NO_valid_trial]

def prepare_features(path,subject,crossValidation=False):
    fs = 250 
    t1 = int(3*fs)
    t2 = int(7*fs)
    T = t2-t1
    X_train, y_train = get_data(subject+1,True,path)
    if crossValidation:
        X_train, X_test, y_train, y_test = train_test_split(
            X_train, y_train, test_size=0.2, random_state=0)
    else:
        X_test, y_test = get_data(subject+1,False,path)

    # prepare training data 	
    N_tr,N_ch,_ =X_train.shape 
    X_train = X_train[:,:,t1:t2].reshape(N_tr,1,N_ch,T)
    y_train_onehot = (y_train-1).astype(int)
    y_train_onehot = to_categorical(y_train_onehot)
    # prepare testing data 
    N_test,N_ch,_ =X_test.shape 
    X_test = X_test[:,:,t1:t2].reshape(N_test,1,N_ch,T)
    y_test_onehot = (y_test-1).astype(int)
    y_test_onehot = to_categorical(y_test_onehot)	
    return X_train,y_train,y_train_onehot,X_test,y_test,y_test_onehot

5. Modified EEGNet

In [4]:
class Constrainedlinear(nn.Linear):
    def _max_norm(self,w,max_val=0.25,eps=1e-8):
        norm = w.norm(2, dim=1, keepdim=True)
        desired = torch.clamp(norm, 0, max_val)
        return w * (desired / (eps + norm))
    def forward(self, input):
        return F.linear(input, self._max_norm(self.weight),self.bias)


class Constrainedlinear111(nn.Linear):
    def _max_norm(self,w,max_val=0.5,eps=1e-8):
        norm = w.norm(2, dim=1, keepdim=True)
        desired = torch.clamp(norm, 0, max_val)
        return w * (desired / (eps + norm))
    def forward(self, input):
        return F.linear(input, self._max_norm(self.weight),self.bias)


class EEGNet(nn.Module):
    def __init__(self, classes_num):
        super(EEGNet, self).__init__()
        self.numC=3

        self.drop_out = 0.2
        
        self.block_1 = nn.Sequential(
            # Pads the input tensor boundaries with zero
            # left, right, up, bottom
            nn.ZeroPad2d((15, 16, 0, 0)),
            nn.Conv2d(
                in_channels=1,          # input shape (1, C, T)
                out_channels=8,         # num_filters
                kernel_size=(1, 32),    # filter size
                bias=False
            ),                          # output shape (8, C, T)
            nn.BatchNorm2d(8)           # output shape (8, C, T)
        )
        
        # block 2 and 3 are implementations of Depthwise Convolution and Separable Convolution
        self.block_2 = nn.Sequential(
            nn.Conv2d(
                in_channels=8,          # input shape (8, C, T)
                out_channels=16,        # num_filters
                kernel_size=(self.numC, 1),    # filter size
                groups=8,
                bias=False
            ),                          # output shape (16, 1, T)
            nn.BatchNorm2d(16),         # output shape (16, 1, T)
            nn.ELU(),
            nn.AvgPool2d((1, 8)),       # output shape (16, 1, T//4)
            nn.Dropout(self.drop_out)   # output shape (16, 1, T//4)
        )
        
        self.block_3 = nn.Sequential(
            nn.ZeroPad2d((7, 8, 0, 0)),
            nn.Conv2d(
               in_channels=16,          # input shape (16, 1, T//4)
               out_channels=16,         # num_filters
               kernel_size=(1, 16),     # filter size
               groups=16,
               bias=False
            ),                          # output shape (16, 1, T//4)
            nn.Conv2d(
                in_channels=16,         # input shape (16, 1, T//4)
                out_channels=16,        # num_filters
                kernel_size=(1, 1),     # filter size
                bias=False
            ),                          # output shape (16, 1, T//4)
            nn.BatchNorm2d(16),         # output shape (16, 1, T//4)
            nn.ELU(),
            nn.AvgPool2d((1, 8)),       # output shape (16, 1, T//32)
            nn.Dropout(self.drop_out)
        )


        self.preluip1 = nn.PReLU()
        self.out = Constrainedlinear((16 * 15), classes_num)
    
    def forward(self, x):
        x = self.block_1(x)
        x = self.block_2(x)
        x = self.block_3(x)
        
        x = x.view(x.size(0), -1)
        x = self.out(x)
        return F.log_softmax(x, dim=1) #x   # return x for visualization



8. Train and Testing

In [7]:
def train(Subject_number):
  #Prepare the raw feature data for training and testing#
  #You can change the "Subject_number" (0-8 corresponding to B01-B09, respectively) to try training and testing between different subjects
  Subject_number=Subject_number
  BATCH_SIZE = 10
  x_train,y_train,y_train_onehot,x_test,y_test,y_test_onehot = prepare_features('',Subject_number,False)
  total_x=np.vstack((x_train,x_test))
  total_y=np.hstack((y_train,y_test))
  # total_y_onehot=np.vstack((y_train_onehot,y_test_onehot))
  index1=np.where(total_y == 1)[0]
  index2=np.where(total_y == 2)[0]
  choice1=np.random.choice(index1,10,replace=False)
  choice2=np.random.choice(index2,10,replace=False)
  choice_total=np.hstack((choice1,choice2))
  x_train=total_x[choice_total]
  x_test= np.delete(total_x, choice_total,0)
  y_train=total_y[choice_total]
  y_test=np.delete(total_y, choice_total)





  x_train=torch.from_numpy(x_train)
  y_train=torch.from_numpy(y_train-1)
  x_test=torch.from_numpy(x_test)
  y_test=torch.from_numpy(y_test-1)
  x_train=x_train.float()
  x_test=x_test.float()
  y_train=y_train.long()
  y_test=y_test.long()
  train_dataset=Data.TensorDataset(x_train,y_train)
  val_dataset=Data.TensorDataset(x_test,y_test)

  #Construct the training Loader
  train_loader= Data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
  test_loader = Data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)


  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  eeg_net = EEGNet(classes_num=2).to(device)

  # summary(eeg_net,(1, 3, 1000))

  nllloss = nn.NLLLoss().to(device) #CrossEntropyLoss = log_softmax + NLLLoss

# optimzer4nn
  optimizer4nn = optim.Adam(eeg_net.parameters(),lr=0.001)

  #Define the number of epochs
  EPOCH=100

#Define the device
  DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  for epoch in range(EPOCH):
    loss_total=[]

  #Update x_i using the gradient descent
    for step, (b_x, b_y) in enumerate(train_loader):
      b_x, b_y = b_x.to(DEVICE), b_y.to(DEVICE)
      output = eeg_net(b_x)
      loss = nllloss(output,b_y) 
      loss_total.append(loss.item())
      optimizer4nn.zero_grad()
      loss.backward()
      optimizer4nn.step()
    if (epoch+1)%100==0:
      print('Epoch: ', epoch+1, '| train loss: %.4f' % (np.mean(loss_total)),len(loss_total))
 
  eeg_net.eval()
  train_out=eeg_net(x_train.to(DEVICE))
  trpre_y = torch.max(train_out.cpu(), 1)[1].data.numpy()
  tra_acc = float((trpre_y == y_train.data.numpy()).astype(int).sum()) / float(y_train.size(0))
  print("Subject "+str(Subject_number+1)+' train accuracy: %.4f' % tra_acc)


  test_out = eeg_net(x_test.to(DEVICE))
  pred_y = torch.max(test_out.cpu(), 1)[1].data.numpy()
  accuracy = float((pred_y == y_test.data.numpy()).astype(int).sum()) / float(y_test.size(0))
  print("Subject "+str(Subject_number+1)+' test accuracy: %.4f' % accuracy)
  print('')
  return accuracy


In [8]:
# Change the subject number 0-8 corrsponding to subject B01-B09; Subject_number=0
Subject_number=2
print("Subject:",Subject_number+1)
train(Subject_number)

Subject: 3
Epoch:  100 | train loss: 0.2615 2
Subject 3 train accuracy: 1.0000
Subject 3 test accuracy: 0.6107



0.6106719367588933